In [437]:
import spotipy
from spotipy import util
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import pandas as pd
import matplotlib.pyplot as plt
import urllib

## REMOVE BEFORE PUSHING ##
CLIENT_ID=
CLIENT_SECRET=



username = 'elw86ve5g5t944wwlef6qyzu3' # Alex Fioto's user id
modify_playlist = 'playlist-modify-public'
user_auth = 'user-top-read'
redirect_uri = 'http://127.0.0.1:8080'

token = util.prompt_for_user_token(username=username,
                                   scope=modify_playlist + ' ' + user_auth,
                                   client_id=CLIENT_ID,
                                   client_secret=CLIENT_SECRET,
                                   redirect_uri=redirect_uri) 

sp = spotipy.Spotify(auth=token)


In [438]:
logslaught = 'spotify:artist:1I471vwcRhqQl6QonGZlen'
grandson = 'spotify:artist:4ZgQDCtRqZlhLswVS6MHN4'
run_the_jewels = 'spotify:artist:4RnBFZRiMLRyZy0AzzTg2C'
hatebreed = 'spotify:artist:17Mb968quDHpjCkIyq30QV'

In [439]:
def get_artist_tracks(sp, artists, n_albums=100,):
    '''
    Function takes in the Spotify URI of one or more artists and returns a Pandas dataframe with Spotify's proprietary audio features.
    
    n_albums: number of albums to fetch per artist
    '''
    
    uris = []
    album_uris = []
    df_list = []
    
    # Checking if user input artists as a list. 
    if type(artists) != list:
        # Rectifying if user did not input list
        artists = [artists]
    
    for artist in artists:
        albums = sp.artist_albums(artist)
        for album in albums['items']:
            album_uris.append(album['uri'])
    
    for uri in album_uris:
        album = sp.album(uri)
        album_name = album['name']
        
        album_artist = ', '.join([artist['name'] for artist in album['artists']])
        
        tracks = sp.album_tracks(uri)
        audio_features_dict = sp.audio_features()
        df = pd.DataFrame.from_dict(tracks['items'])
        audio_features_df = pd.DataFrame.from_dict(sp.audio_features(list(df['uri'])))
        audio_features_df['track_name'] = df['name']
        audio_features_df['album_name'] = album_name
        audio_features_df['artist_name'] = album_artist
        df_list.append(audio_features_df)
    
    df = pd.concat(df_list)
    df.reset_index(inplace=True, drop=True)
    df.drop_duplicates()
    return df

In [440]:
def get_user_tracks(sp, limit=10, time_range='medium_term'):
    user_tracks = sp.current_user_top_tracks(limit=limit, time_range=time_range)
    audio_features =  sp.audio_features([item['uri'] for item in user_tracks['items']])
    names = [item['name'] for item in user_tracks['items']]
    df = pd.DataFrame.from_dict(audio_features)
    df['track_name'] = names
    return df

In [441]:
user_df = get_user_tracks(sp = sp)

In [442]:
user_df.head(2)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name
0,0.817,0.814,2,-3.985,1,0.185,0.3200,0.00000000,0.1520,0.247,109.994,audio_features,5rLyYxZNzca00ENADO9m54,spotify:track:5rLyYxZNzca00ENADO9m54,https://api.spotify.com/v1/tracks/5rLyYxZNzca0...,https://api.spotify.com/v1/audio-analysis/5rLy...,196800,4,When I Grow Up
1,0.701,0.525,1,-7.938,0,0.382,0.0353,0.00000255,0.0615,0.385,75.693,audio_features,5jQYkYhoOlBW4vJ2l4TCxl,spotify:track:5jQYkYhoOlBW4vJ2l4TCxl,https://api.spotify.com/v1/tracks/5jQYkYhoOlBW...,https://api.spotify.com/v1/audio-analysis/5jQY...,152253,4,"Blockbuster Night, Pt. 1"


In [443]:
tracks = sp.current_user_top_tracks()

In [444]:
tracks['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6fOMl44jA4Sp5b9PpYCkzz'},
  'href': 'https://api.spotify.com/v1/artists/6fOMl44jA4Sp5b9PpYCkzz',
  'id': '6fOMl44jA4Sp5b9PpYCkzz',
  'name': 'NF',
  'type': 'artist',
  'uri': 'spotify:artist:6fOMl44jA4Sp5b9PpYCkzz'}]

In [445]:
def user_tracks_dataframe(spotify, limit=10):
    '''
    This function takes in a spotify user authenication object and a limit of songs.
    Return a pandas data frame with users top tracks (names and artists) 
    within short, medium and long-term time-ranges

    '''
    terms = ['short_term', 'medium_term', 'long_term']
    df_list = []
    for term in terms:
        tracks = spotify.current_user_top_tracks(limit=limit, time_range=term)
        track_names = [track['name'] for track in tracks['items']]
        artist_list = []
        for track in tracks['items']:
            artists = ', '.join([artist['name'] for artist in track['artists']])
            artist_list.append(artists)
        df = pd.DataFrame(list(zip(track_names, artist_list)), 
                          columns = ['track_name', 'artist_name'])
        df['time_range'] = term
        df_list.append(df)
    df = pd.concat(df_list)
    df.reset_index(inplace=True, drop=True)
    return df
        

In [446]:
def get_playlist_embeddings(spotify):
    '''
    Takes in user authentication object and returns HTML embedding codes for user playlists
    '''
    playlists = spotify.current_user_playlists()
    playlist_ids = [playlist['id'] for playlist in playlists['items']]
    embed_code = '<iframe src="https://open.spotify.com/embed/playlist/####" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'
    codes = ' '.join([embed_code.replace('####', playlist_id) for playlist_id in playlist_ids])
    return codes

In [431]:
%%html
'<iframe src="https://open.spotify.com/embed/playlist/2rX148mot1TpEDPbJdrqjs" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe> <iframe src="https://open.spotify.com/embed/playlist/36IMauEyDsOmc6NoJ1ECPM" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe> <iframe src="https://open.spotify.com/embed/playlist/7gX4gtuKBJyZS8qll6W6ud" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe> <iframe src="https://open.spotify.com/embed/playlist/04xlIANGeecMR5UNd8RBic" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'

In [212]:
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
pd.set_option("display.precision", 14)

In [447]:
def compare_songs(artist_df, user_df):
    '''
    Takes in a dataframe of artist tracks and audio feautes, and a dataframe of user top tracks. 
    Returns tracks in artist_df that are most alligned with user tracks.
    '''
    
    band_track_names = artist_df['track_name']
    user_track_names = user_df['track_name']
    
    df_numeric = artist_df.loc[:, artist_df.columns[:11]]
    user_df_numeric = user_df.loc[:, user_df.columns[:11]]
    
    recs = pairwise_distances(df_numeric, user_df_numeric, metric='cosine')
    rec_df = pd.DataFrame(recs, columns=user_track_names, index=band_track_names)    
    rec_df = 1 - rec_df
    rec_df.drop_duplicates(inplace=True)
    
    song_list = []
    
    for user_track in rec_df.columns:
        max_cosine = rec_df[user_track].max()
        song_name = rec_df[rec_df[user_track] == max_cosine].index[0]
        song_uri = list(artist_df.loc[artist_df['track_name'] == song_name, 'uri'])[0]
        #print(f'{user_track}------ closest to ---------> {song_name}')
        song_list.append((song_name, song_uri))
        
    uri_list = []
    [uri_list.append(track[1]) for track in song_list if track[1] not in uri_list]
    
    return uri_list

# Test out Functions!
get_artist_tracks
get_user_tracks
compare_songs

In [448]:
glass_animals = 'spotify:artist:4yvcSjfu4PC0CYQyLy4wSq'

In [449]:
artist_tracks = get_artist_tracks(sp, artists = glass_animals )
user_tracks = get_user_tracks(sp, limit=2, time_range='long_term')

In [352]:
artist_tracks.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,artist_name
0,0.253,0.352,8,-11.509,1,0.0357,0.8830,0.001910,0.0668,0.099,...,audio_features,72e9a2yjWAAzMQoHxNb5f8,spotify:track:72e9a2yjWAAzMQoHxNb5f8,https://api.spotify.com/v1/tracks/72e9a2yjWAAz...,https://api.spotify.com/v1/audio-analysis/72e9...,203492,5,Dreamland,Dreamland (+ Bonus Levels),Glass Animals
1,0.845,0.583,9,-5.948,0,0.0773,0.1140,0.000536,0.1000,0.842,...,audio_features,5duuw7SyxEPJQmXrMmqNPw,spotify:track:5duuw7SyxEPJQmXrMmqNPw,https://api.spotify.com/v1/tracks/5duuw7SyxEPJ...,https://api.spotify.com/v1/audio-analysis/5duu...,200584,4,Tangerine,Dreamland (+ Bonus Levels),Glass Animals
2,0.000,0.138,10,-23.307,0,0.0000,0.9640,0.000000,0.0000,0.000,...,audio_features,6zSnjpObDWJA3TgzoQjbyj,spotify:track:6zSnjpObDWJA3TgzoQjbyj,https://api.spotify.com/v1/tracks/6zSnjpObDWJA...,https://api.spotify.com/v1/audio-analysis/6zSn...,7156,0,((home movie: 1994)),Dreamland (+ Bonus Levels),Glass Animals
3,0.705,0.487,0,-9.456,0,0.0587,0.0837,0.028000,0.3030,0.340,...,audio_features,2PLlKITOpHpLpONfReqx3k,spotify:track:2PLlKITOpHpLpONfReqx3k,https://api.spotify.com/v1/tracks/2PLlKITOpHpL...,https://api.spotify.com/v1/audio-analysis/2PLl...,234866,4,Hot Sugar,Dreamland (+ Bonus Levels),Glass Animals
4,0.000,0.206,5,-21.272,1,0.0000,0.9360,0.032200,0.6760,0.000,...,audio_features,4vTkS9R8MIZqLcxx2pCtHT,spotify:track:4vTkS9R8MIZqLcxx2pCtHT,https://api.spotify.com/v1/tracks/4vTkS9R8MIZq...,https://api.spotify.com/v1/audio-analysis/4vTk...,13626,0,((home movie: btx)),Dreamland (+ Bonus Levels),Glass Animals


In [383]:
song_list, rec_df = compare_songs(artist_tracks, user_tracks)

ValueError: too many values to unpack (expected 2)

In [381]:
song_list

['spotify:track:09ml9rVHNtfvr6jJZPrcPX',
 'spotify:track:3mRBPZbnwsmddN0JH5BNVl',
 'spotify:track:7a9wmrYMkvjS1eEGzQPaLp',
 'spotify:track:0BHStoRqbogddXgezqumz3',
 'spotify:track:1C29ru8rzgL83YXrGVOmUs',
 'spotify:track:29c4JqIjWrAckKvvPnTeUC',
 'spotify:track:65PsOtxOpgaGvjztMGzoj2',
 'spotify:track:1FhvSbjZmQQ5qeKXo6JilC']

In [454]:
tracks = get_artist_tracks(sp, artists='spotify:artist:3bcLBxvaI7GsBzGp3WHnwQ')
user_tracks = get_user_tracks(sp)

In [453]:
tracks

499

In [456]:
len(user_tracks)

10

In [459]:
compare_songs(tracks, user_tracks)

['spotify:track:2IF8sqTWsKptMllsYglkeK',
 'spotify:track:0POBjbE1ovNWInRDVQbREC',
 'spotify:track:7KsGeiA9C4x63MCJcdpL12',
 'spotify:track:1jIluyycudIsv7OrURzBBU',
 'spotify:track:0BhFrPI9q21LadxoIgT2zf',
 'spotify:track:76g5npBAaPI8RDQETVehR7',
 'spotify:track:0OkNhfAJPFb54zavGW0RAN',
 'spotify:track:4e2rhIyiyAeWuU23k0kMFC',
 'spotify:track:7DFTzN8Tg1U6zIYn8rgL9y']